Yahoo finance python extension testing 

In [ ]:
import yfinance as yf 
dat = yf.Ticker("ADEN")
eu = yf.Market("EUROPE")


In [ ]:
eu.summary

Scraping tests 

In [ ]:
import requests
import pandas as pd
import json

In [ ]:
def get_income_statement(ticker): 
    url = f"https://marketplace.financialmodelingprep.com/public/income-statement/{ticker}?period=annual&limit=4&apikey="
        
    response = requests.get(url).json()

    with open("test.json", "w") as f:
        json.dump(response, f)


get_income_statement("UHR.SW")

In [ ]:
    
companies = pd.read_csv("data\equity_issuers.csv", delimiter=";", encoding = "ISO-8859-1")
    

In [ ]:
for i, row in companies.iterrows():
    name = row["Company"]
    ticker = row["Symbol"] + ".SW"
    
    
    print("The ticker for " + name + " is " + ticker)

    

Data Prep

In [ ]:
import numpy as np
import pandas as pd
import os
import glob

In [ ]:

df = pd.read_json(r"data\balance_sheets\Zug Estates Holding AG_balance_sheet_statement.json")

df

In [ ]:
# creating dataset with all available balance sheets

balance_sheets = pd.DataFrame()

file_list = glob.glob(r".\data\balance_sheets\*.json")

dfs = [] # an empty list to store the data frames
for file in file_list:
    data = pd.read_json(file)
    dfs.append(data)

balance_sheets = pd.concat(dfs, ignore_index=True)

balance_sheets.to_csv("datasets/balance_sheets.csv")


In [ ]:
# creating dataset with all available cash flow statements

cash_flows = pd.DataFrame()

file_list = glob.glob(rf".\data\cash_flow_statements\*.json")

dfs = [] # an empty list to store the data frames
for file in file_list:
    data = pd.read_json(file)
    dfs.append(data)
    
cash_flows = pd.concat(dfs, ignore_index=True)

cash_flows.to_csv("datasets/cash_flows.csv")

In [ ]:
# creating dataset with all available income statements

income_statements = pd.DataFrame()

file_list = glob.glob(rf".\data\income_statements\*.json")

dfs = []
for file in file_list:
    data = pd.read_json(file)
    dfs.append(data)
    
income_statements = pd.concat(dfs, ignore_index=True)

income_statements.to_csv("datasets/income_statements.csv")


dataset playground

In [ ]:
import pandas as pd

In [ ]:


df = pd.read_csv("datasets/balance_sheets.csv")

print(df.head())

In [ ]:
# INFO
print("INFO\n")
print(df.info())
print("*"*50+"\n")

# STAT SUMMARY
print("STAT SUMMARY\n")
print(df.describe())
print("*"*50+"\n")
# MISING VALUES
print("MISSING VALUES\n")
print(df.isnull().sum())

In [ ]:
df.drop(columns='cik')

In [ ]:
# calculating the altman z-score
df["z_score"] = 1.2*df["totalCurrentAssets"]/df["totalAssets"] + 1.4*df["retainedEarnings"]/df["totalAssets"] + 3.3*df["???ebit"]/df["totalAssets"] + 0.6*df["???marketCap"]/df["totalLiabilities"] + 0.999*df["???totalRevenue"]/df["totalAssets"]

In [ ]:
# read cash flows
df_cf = pd.read_csv("datasets/cash_flow_statements.csv")

print(df_cf.head())

In [ ]:
# INFO
print("INFO\n")
print(df_cf.info())
print("*"*50+"\n")

# STAT SUMMARY
print("STAT SUMMARY\n")
print(df_cf.describe())
print("*"*50+"\n")
# MISING VALUES
print("MISSING VALUES\n")
print(df_cf.isnull().sum())

In [ ]:
# read income statements
df_is = pd.read_csv("datasets/income_statements.csv")

print(df_is.head())

In [ ]:
# INFO
print("INFO\n")
print(df_is.info())
print("*"*50+"\n")

# STAT SUMMARY
print("STAT SUMMARY\n")
print(df_is.describe())
print("*"*50+"\n")
# MISING VALUES
print("MISSING VALUES\n")
print(df_is.isnull().sum())

In [ ]:
df_full = pd.read_csv("datasets/full_raw_dataset.csv")

In [ ]:
# INFO
print("INFO\n")
print(df_full.info())
print("*"*50+"\n")

# STAT SUMMARY
print("STAT SUMMARY\n")
print(df_full.describe())
print("*"*50+"\n")

# MISING VALUES
print("MISSING VALUES\n")
print(df_full.isnull().sum())

In [ ]:
x = df_full.columns.tolist()
for each in x:
    print(each)

In [ ]:
df_full_sample = df_full.head()
df_full_sample = df_full_sample.transpose()

df_full_sample.to_csv("raw_cleanup_guide.csv")

Cleanup of full_raw_dataset
refer to raw_cleanup_guide for details

cleaning duplicates and unnecessary columns

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("datasets/full_raw_dataset.csv")

# drop unnecessary columns 
df = df.drop(columns=["Unnamed: 0.1", "Unnamed: 0_x", "Unnamed: 0_y", "Unnamed: 0", "cik", "cik_y", "cik_x", "fillingDate", "fillingDate_x", "fillingDate_y",
             "acceptedDate", "acceptedDate_x", "acceptedDate_y", "link", "link_x", "link_y", "finalLink", "finalLink_x", "finalLink_y", "distressed_y", "distressed_x", "period", "period_x", "period_y"])

# rename currency columns
df.rename(columns={"reportedCurrency_x": "currency_balance_sheet", "reportedCurrency_y": "currency_cash_flow_statement", "reportedCurrency": "currency_income_statement"}, inplace=True)
    
# combine calendarYears to one column
df['Year'] = df['calendarYear'].combine_first(df['calendarYear_x']).combine_first(df['calendarYear_y'])
df = df.drop(columns=["calendarYear", "calendarYear_x", "calendarYear_y"])

# rename inventory columns
df.rename(columns={"inventory_x": "inventory_balance_sheet", "inventory_y": "inventory_cash_flow_statement"}, inplace=True)

# rename netIncome columns
df.rename(columns={"netIncome_x": "netIncome_cash_flow_statement", "netIncome_y": "netIncome_income_statement", "netIncomeRatio": "netIncomeRatio_income_statement"}, inplace=True)

# rename depreciationAndAmortization columns
df.rename(columns={"depreciationAndAmortization_x": "depreciationAndAmortization_cash_flow_statement", "depreciationAndAmortization_y": "depreciationAndAmortization_income_statement"}, inplace=True)

df.to_csv("datasets/full_cleaned_dataset.csv")
    

In [ ]:
# Manuel duplicate checks

#df['currencyComp_orig_x'] = np.where(df['reportedCurrency'] != df['reportedCurrency_x'], 1, 0)
#df['currencyComp_orig_y'] = np.where(df['reportedCurrency'] != df['reportedCurrency_y'], 1, 0)
df['dna_comp'] = np.where(df['depreciationAndAmortization_x'] != df['depreciationAndAmortization_y'], 1, 0)

df.to_csv('temp.csv')


In [ ]:
# create sample dataset to have an overview/guide for the analysis
import pandas as pd
df = pd.read_csv("datasets/full_cleaned_dataset.csv")
df_sample = df.sample(1)
df_sample = df_sample.transpose()
df_sample.to_csv("sample_dataset.csv")

webscraper tests for companiesmarketcap

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [ ]:
response = requests.get(url)   

if response.status_code != 200:
    print("Failed to connect")
 
soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find('table', {'class': 'table'})

headers = []   
header_row = table.find('thead').find('tr') if table.find('thead') else table.find('tr')
for th in header_row.find_all('th'):
    headers.append(th.text.strip()) 
    
    
rows = []
for row in table.find_all('tr'):
    cells = row.find_all(['td', 'th'])
    if len(cells) > 0:
        rows.append([cell.text.strip() for cell in cells])
        
df = pd.DataFrame(rows)
    
df.to_csv("data/market_caps/abb_marketcap.csv", header=None, index=None)



In [ ]:
companies = pd.read_csv("data\equity_issuers.csv", delimiter=";", encoding = "ISO-8859-1") #encoding is necessary to match the encoding of the source file
url = "https://companiesmarketcap.com/search.do"
responses = []


for i, row in companies.iterrows():
    ticker = row["Symbol"]
    name = row["Company"]
    payload = {
        'query': f"{ticker}"
    }
    
    response = requests.post(url, data = payload)
    responses.append(response)   
    
df = pd.DataFrame(responses)

df.to_json("market_cap_tickersearch.json")

In [ ]:
import glob
filelist = glob.glob("data\market_caps\*.csv")

for i in filelist:
    x = i.split("_")[1].replace("caps\\", "")
    print(x)

In [19]:
# merge marketcap csv into full dataset
import pandas as pd

df = pd.read_csv("datasets/full_cleaned_dataset.csv")
df_mc = pd.read_csv("datasets/market_caps.csv")
df_mc = df_mc.drop(columns=["Change"])

df = pd.merge(df, df_mc, on=["symbol", "Year"], how= "left") 

df.to_csv("temp.csv")

In [8]:
import pandas as pd
import numpy as np

df = pd.read_csv("../datasets/1std_dataset.csv")

df['qwe'] = np.where(df['z_score'] >= 3.0, 1, 0)

print(df['qwe'].value_counts())

qwe
1    345
0    322
Name: count, dtype: int64
